In [100]:
import pandas as pd
x_train = pd.read_csv('data/train/subtaskA_data_all.csv')
y_train = pd.read_csv('data/train/subtaskA_answers_all.csv', header=None)
x_test = pd.read_csv('data/trial/taskA_trial_data.csv')
y_test = pd.read_csv('data/trial/taskA_trial_answer.csv', header=None)
y_train = y_train.rename(columns={0 : 'id',1: 'invalid_sent'})
y_test = y_test.rename(columns={0 : 'id',1: 'invalid_sent'})
xy = x_train.merge(y_train)
items = []
for row in xy.itertuples():
    if row.invalid_sent == 0:
        items.append({'id' : row.id, 'sent' : row.sent0, 'isvalid' : 0.0})
        items.append({'id' : row.id, 'sent' : row.sent1, 'isvalid' : 1.0})
    else:
        items.append({'id' : row.id, 'sent' : row.sent0, 'isvalid' : 1.0})
        items.append({'id' : row.id, 'sent' : row.sent1, 'isvalid' : 0.0})
df = pd.DataFrame(items)


In [101]:
xytest = x_test.merge(y_test)
items = []
for row in xytest.itertuples():
    if row.invalid_sent == 0:
        items.append({'id' : row.id, 'sent' : row.sent0, 'isvalid' : 0.0})
        items.append({'id' : row.id, 'sent' : row.sent1, 'isvalid' : 1.0})
    else:
        items.append({'id' : row.id, 'sent' : row.sent0, 'isvalid' : 1.0})
        items.append({'id' : row.id, 'sent' : row.sent1, 'isvalid' : 0.0})
testdf = pd.DataFrame(items)

In [3]:
##################### Logistic Regression #########################

In [4]:
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

ps = PorterStemmer()
stop_words = set(stopwords.words('english')) 
def preprocess(x):
    x = x.lower()
    tokens = word_tokenize(x)
    filtered_tokens = [w for w in tokens if not w in stop_words] 
    return ' '.join([ps.stem(t) for t in filtered_tokens])
pdf = df.sent.apply(preprocess)
testpdf = testdf.sent.apply(preprocess)

[nltk_data] Downloading package punkt to /Users/goelprat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/goelprat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=0, lowercase=False, max_features=3000)
vectorizer.fit(pdf)
#vectorizer2 = CountVectorizer(min_df=0, lowercase=False, max_features=3000)
#vectorizer2.fit(testpdf)
len(vectorizer1.vocabulary_.keys())

3000

In [13]:
X_train = vectorizer.transform(pdf)
X_test = vectorizer.transform(testpdf)
Y_train = df.isvalid
Y_test = testdf.isvalid

In [21]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, Y_train)

/Users/goelprat/miniconda3/envs/fastai/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
score = classifier.score(X_test, Y_test)
score

0.5774369124195943

In [ ]:
### Answer 5
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
### Answer 3
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

def create_model(optimizer='sgd'):
    model = Sequential()
    model.add(Dense(units = 1, activation = 'sigmoid', input_dim = 3000))
    # Describe the loss and how it is optimized
    model.compile(loss = 'mean_squared_error', optimizer = optimizer, metrics = ['accuracy'])
    return model

def grid_search():
    optimizers = []
    model = KerasClassifier(build_fn=create_model)
    optimizer = ['SGD', 'Adam', 'RMSProp']
    param_grid = dict(epochs=[20, 40, 60], batch_size=[20, 40, 60], optimizer=optimizer)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
    grid_result = grid.fit(x_train, y_train)

    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
        
model = create_model(optimizer='adam')
model.fit(X_train, Y_train, epochs=40, batch_size=40)


In [26]:
model.evaluate(X_test, Y_test)

4042/4042 [==============================] - 0s 57us/step


[0.24135216026539735, 0.5821375846862793]

In [4]:
##################### Playing with fastai library #########################

In [147]:
from fastai.text import *
class CommonSense_NN(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        #self.emb = nn.Embedding(vocab_size, 300) 
        self.lin1 = nn.Linear(vocab_size, 300, bias=True)
        self.lin2 = nn.Linear(300, 1, bias=True)
        
    def forward(self, x):
        #x = xb.view(-1,784) 
        #import ipdb; ipdb.set_trace()
        #x = self.emb(x)
        x = F.relu(self.lin1(x.float()))
        x = torch.sigmoid(self.lin2(x))
        return x.squeeze()

(4042, 1)

In [ ]:
from fastai.text import *
class CommonSense_Logistic(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.lin1 = nn.Linear(vocab_size, 1, bias=True)

    def forward(self, x):
        #import ipdb; ipdb.set_trace()
        x = F.sigmoid(self.lin1(x.float()))
        return x.squeeze()

In [ ]:
from fastai.text.data import TokenizeProcessor, NumericalizeProcessor
class BoWProcessor(NumericalizeProcessor):
    max_vocab = 2000
    min_freq=3
    def process_one(self,item):
        if isinstance(item, EmptyLabel): return item
        import ipdb; ipdb.set_trace()
        vsize = len(self.vocab.itos)
        res = np.zeros(vsize, dtype=np.int64)
        for x in item:
            res[self.vocab.stoi[x]] = 1
        if res is None: self.warns.append(str(item))
        return res
    
    def process(self, ds):
        if self.vocab is None: self.vocab = Vocab.create(ds.items, 2000, 3)
        ds.vocab = self.vocab
        super().process(ds)

In [ ]:
tlist = TextList.from_df(df, cols=['sent'], processor=[TokenizeProcessor(include_bos=False),BoWProcessor()])
split_data = tlist.split_by_rand_pct()
labeled_data = split_data.label_from_df(cols='isvalid')
textdb = labeled_data.databunch()
len(textdb.train_ds.x.vocab.itos)

In [ ]:
learner = Learner(data=textdb, model=CommonSense_NN(vocab_size=len(textdb.train_ds.x.vocab.itos)), loss_func=nn.MSELoss())
learner.lr_find()
learner.recorder.plot()
learner.fit_one_cycle(5,1e-06)


In [ ]:
##################### Using BERT to get document embeddings #########################

In [3]:
from nltk.tokenize import sent_tokenize, word_tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
unk = 0
for sent in df.sent:
    sent = sent.lower()
    tokens = word_tokenize(sent)
    for token in tokens :
        if not tokenizer.vocab.get(token):
            unk += 1
print(unk)

NameError: name 'BertTokenizer' is not defined

In [4]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)
bertdf = df.sent.apply(lambda x : "[CLS] " + x + " [SEP]")
testbertdf = testdf.sent.apply(lambda x : "[CLS] " + x + " [SEP]")
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_tensors = []
for sent in bertdf:
    # Tokenize input
    tokenized_text = tokenizer.tokenize(sent)
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    train_tensors.append(tokens_tensor)
    
test_tensors = []
for sent in testbertdf:
    # Tokenize input
    tokenized_text = tokenizer.tokenize(sent)
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    test_tensors.append(tokens_tensor)

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/goelprat/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [60]:
train_tensors[9]

tensor([[  101,  1037, 21025, 27528,  7959,  2003,  1037,  2711,  1012,   102]])

In [61]:
tokenizer.convert_ids_to_tokens([  101,  1037, 21025, 27528,  7959,  2003,  1037,  2711,  1012,   102])

['[CLS]', 'a', 'gi', '##raf', '##fe', 'is', 'a', 'person', '.', '[SEP]']

In [5]:
import torch.nn.functional as F
import numpy as np
import math
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

X_test = np.zeros((4042, 768))
with torch.no_grad():
    for i in range(4042):
        tensor = test_tensors[i]
        outputs = model(tensor)
        embeddings = outputs[0].squeeze()
        doc = torch.mean(embeddings.squeeze(), axis=0)
        X_test[i] = doc
        
X = np.zeros((20000, 768))
with torch.no_grad():
    for i in range(20000):
        tensor = train_tensors[i]
        outputs = model(tensor)
        embeddings = outputs[0].squeeze()
        doc = torch.mean(embeddings.squeeze(), axis=0)
        X[i] = doc

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /Users/goelprat/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

INFO:transformers.modeling_utils:loading weights file https://s3.amazona

KeyboardInterrupt: 

In [52]:
Y = df.isvalid
Y_test = testdf.isvalid

In [53]:
Y.shape

(20000,)

In [54]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X, Y)

/Users/goelprat/miniconda3/envs/fastai/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [57]:
score = classifier.score(X_test, Y_test)
score

0.6358238495794162

In [11]:
##################### Using BERT to get perplexity #########################

In [3]:
bertdf = df.sent.apply(lambda x : "[CLS] " + x + " [SEP]")
bertdf

0        [CLS] He poured orange juice on his cereal. [SEP]
1                [CLS] He poured milk on his cereal. [SEP]
2                             [CLS] He drinks apple. [SEP]
3                              [CLS] He drinks milk. [SEP]
4                        [CLS] Jeff ran a mile today [SEP]
                               ...                        
19995    [CLS] Harry went to the barbershop to have his...
19996          [CLS] Reilly is sleeping on the couch [SEP]
19997         [CLS] Reilly is sleeping on the window [SEP]
19998                 [CLS] I have a desk on my lamp [SEP]
19999                 [CLS] I have a lamp on my desk [SEP]
Name: sent, Length: 20000, dtype: object

In [4]:
testbertdf = testdf.sent.apply(lambda x : "[CLS] " + x + " [SEP]")
testbertdf

0          [CLS] he put an elephant into the fridge [SEP]
1             [CLS] he put a turkey into the fridge [SEP]
2       [CLS] my sister eats an apple after breakfast ...
3       [CLS] my sister eats a stone after breakfast e...
4           [CLS] money can be used for buying cars [SEP]
                              ...                        
4037    [CLS] people have to hold onto their shoes bec...
4038    [CLS] Rebecca wakes up because she takes sleep...
4039    [CLS] Rebecca wakes up because she sets an ala...
4040          [CLS] people bleed when they feel hot [SEP]
4041          [CLS] people sweat when they feel hot [SEP]
Name: sent, Length: 4042, dtype: object

In [9]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_tensors = []
for sent in bertdf:
    # Tokenize input
    tokenized_text = tokenizer.tokenize(sent)
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    train_tensors.append(tokens_tensor)
    
test_tensors = []
for sent in testbertdf:
    # Tokenize input
    tokenized_text = tokenizer.tokenize(sent)
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    test_tensors.append(tokens_tensor)

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/goelprat/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [13]:
test_tensors

tensor([[  101,  2002,  2404,  2019, 10777,  2046,  1996, 16716,   102]])

In [82]:
from transformers import BertForNextSentencePrediction
model = BertForNextSentencePrediction.from_pretrained('bert-base-uncased')

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /Users/goelprat/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

INFO:transformers.modeling_utils:loading weights file https://s3.amazona

In [99]:
import torch.nn.functional as F
import math
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
loss_fct = torch.nn.CrossEntropyLoss()

def run_tokens(s):
    print(s)
    tokenized_text = tokenizer.tokenize(s)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_ids = [0] * tokens_tensor.shape[1]
    segments_tensors = torch.tensor([segments_ids])
    prediction = model(tokens_tensor, token_type_ids=segments_tensors)
    predictions = prediction[0]
    predicted_tokens = []
    for i in range(tokens_tensor.shape[1]):
        predicted_index = torch.argmax(predictions[0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        predicted_tokens.append(predicted_token)
    return predicted_tokens

def run_perp(s):
    print(s)
    tokenized_text = tokenizer.tokenize(s)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_ids = [0] * tokens_tensor.shape[1]
    segments_tensors = torch.tensor([segments_ids])
    prediction = model(tokens_tensor, token_type_ids=segments_tensors)
    loss = loss_fct(prediction[0].squeeze(),tokens_tensor.squeeze()).data 
    return math.exp(loss)

def run(sents, mode):
    model.eval()
    f = run_perp if mode == 'perp' else run_tokens
    out = []
    with torch.no_grad():
        for sent in sents:
            out.append(f(sent))
    return out

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /Users/goelprat/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

INFO:transformers.modeling_utils:loading weights file https://s3.amazona

In [100]:
sents = ['[CLS] I jump across a hurdle . [SEP]', '[CLS] I jump across a mountain . [SEP]']
run(sents, 'perp')

[CLS] I jump across a hurdle . [SEP]
[CLS] I jump across a mountain . [SEP]


[73.9704109439202, 69.66865978242653]

In [102]:
sents = []
for s in testdf.sent:
    sents.append('[CLS] ' + s + ' . [SEP]')

In [103]:
perps = run(sents, 'perp')

[CLS] he put an elephant into the fridge . [SEP]
[CLS] he put a turkey into the fridge . [SEP]
[CLS] my sister eats an apple after breakfast every day . [SEP]
[CLS] my sister eats a stone after breakfast every day . [SEP]
[CLS] money can be used for buying cars . [SEP]
[CLS] money can be used for buying stars . [SEP]
[CLS] New York is located in the northeastern part of USA . [SEP]
[CLS] USA is located in the northeastern part of New York . [SEP]
[CLS] a man can better see stars and the moon in daytime . [SEP]
[CLS] a man can hardly see stars and the moon in daytime . [SEP]
[CLS] he was sent to a restaurant for treatment after a car crash . [SEP]
[CLS] he was sent to a hospital for treatment after a car crash . [SEP]
[CLS] his mother became satisfied after he got good grades in the math exam . [SEP]
[CLS] his mother became angry after he got good grades in the math exam . [SEP]
[CLS] cans are usually made of gold . [SEP]
[CLS] cans are usually made of aluminum . [SEP]
[CLS] I put my de

[CLS] the earth is a planet of the solar system . [SEP]
[CLS] we stopped at the power plant because we needed gasoline . [SEP]
[CLS] we stopped at the filling station because we needed gasoline . [SEP]
[CLS] Lily goes to the cinema to watch a movie . [SEP]
[CLS] Lily goes to the cinema to listen to a concert . [SEP]
[CLS] centigrade is a measure of acidity . [SEP]
[CLS] centigrade is a measure of temperature . [SEP]
[CLS] the ancients used a compass as a guidance . [SEP]
[CLS] the ancients used a satellite as a guidance . [SEP]
[CLS] the fire was caused by a lighted cigarette . [SEP]
[CLS] the fire was caused by a glass of water . [SEP]
[CLS] December is the 12th month of a year . [SEP]
[CLS] December is the 13th month of a year . [SEP]
[CLS] a prince is a piece used in the game of chess . [SEP]
[CLS] a king is a piece used in the game of chess . [SEP]
[CLS] maid is an important member of a family . [SEP]
[CLS] mother is an important member of a family . [SEP]
[CLS] he opens the door w

[CLS] it is really painful for a mother to give birth to her baby . [SEP]
[CLS] it is really painful for a father to give birth to his baby . [SEP]
[CLS] they sleep in a hotel while camping in the forest . [SEP]
[CLS] they sleep in a tent while camping in the forest . [SEP]
[CLS] i water my mp3 player two times a week . [SEP]
[CLS] i charge my mp3 player two times a week . [SEP]
[CLS] Mongolia is mass surrounded by water . [SEP]
[CLS] the UK is mass surrounded by water . [SEP]
[CLS] she wears glasses because of nearsightedness . [SEP]
[CLS] she wears clothes because of nearsightedness . [SEP]
[CLS] Tim buys a lock to prevent his bike from being stolen . [SEP]
[CLS] Tim buys a pair of socks to prevent his bike from being stolen . [SEP]
[CLS] the doctor treats his patients carefully . [SEP]
[CLS] the cook treats his patients carefully . [SEP]
[CLS] the Titanic struck an iceberg and sank . [SEP]
[CLS] the Titanic struck a life buoy and sank . [SEP]
[CLS] he paints the ceiling with cola . 

[CLS] When you smile at others, they will cry to you . [SEP]
[CLS] His favorite color is purple, the same color as the sky . [SEP]
[CLS] His favorite color is blue, the same color as the sky . [SEP]
[CLS] Jenny ordered a hot dog in a pet store . [SEP]
[CLS] Jenny ordered a hot dog in a restaurant . [SEP]
[CLS] I dried my hair before using shampoo . [SEP]
[CLS] I wet my hair before using shampoo . [SEP]
[CLS] The man in black cured him with a bullet . [SEP]
[CLS] The man in black killed him with a bullet . [SEP]
[CLS] She boiled the egg and the egg turned into liquid . [SEP]
[CLS] She boiled the egg and the egg turned into solid . [SEP]
[CLS] Elisa was terrified bitten by a snake . [SEP]
[CLS] Elisa was terrified bitten by a book . [SEP]
[CLS] In the dark, I turn on the light to read a book . [SEP]
[CLS] In the dark, I turn off the light to read a book . [SEP]
[CLS] Atlantic is the biggest continent in the world . [SEP]
[CLS] Asia is the biggest continent in the world . [SEP]
[CLS] The 

[CLS] Cancer can bring happiness to us . [SEP]
[CLS] people can feel pain . [SEP]
[CLS] people can only feel happiness . [SEP]
[CLS] Fish can live in a land without water . [SEP]
[CLS] Fish need water to live . [SEP]
[CLS] Computers can eat food . [SEP]
[CLS] Computers are powered by electricity . [SEP]
[CLS] Everyone lives in the country . [SEP]
[CLS] Some people live in the country . [SEP]
[CLS] People are able to cheat . [SEP]
[CLS] people are only able to be honest . [SEP]
[CLS] If you want to dig a hole then you should use a shovel . [SEP]
[CLS] If you want to dig a hole then you should use the spoon . [SEP]
[CLS] Soldiers will be punished for bravery . [SEP]
[CLS] Soldiers can get medals for bravery . [SEP]
[CLS] Families shouldn't visit relatives during the holidays . [SEP]
[CLS] Families should visit relatives during the workdays . [SEP]
[CLS] I can hold up a car . [SEP]
[CLS] I can hold up a can of soda . [SEP]
[CLS] Everyone is good-looking . [SEP]
[CLS] Most people want to b

...

[CLS] people go to jail because they sinned . [SEP]
[CLS] I saw snow in South Africa in June . [SEP]
[CLS] I saw snow in Singapore in June . [SEP]
[CLS] an umbrella can help you keep warm in snowy days . [SEP]
[CLS] a thicker cloth can help you keep warm in snowy days . [SEP]
[CLS] the plane is a useful transportation tool in campus . [SEP]
[CLS] the bicycle is a useful transportation tool in campus . [SEP]
[CLS] fans went mad when their team scored a goal . [SEP]
[CLS] fans were sad when their team scored a goal . [SEP]
[CLS] I got a scholarship for cheating in the exam . [SEP]
[CLS] I was fined for cheating in the exam . [SEP]
[CLS] the hot weather made me keep bleeding . [SEP]
[CLS] the hot weather made me keep sweating . [SEP]
[CLS] you don't need to try out products when you shop online . [SEP]
[CLS] you can't try out products when you shop online . [SEP]
[CLS] reading can make driving a joyful thing . [SEP]
[CLS] music can make driving a joyful thing . [SEP]
[CLS] everyone said I

[CLS] he found he was gay when he fell in love with a man . [SEP]
[CLS] weighing an elephant is very difficult because of its heavy weight . [SEP]
[CLS] weighing an elephant is very difficult because of its cute appearance . [SEP]
[CLS] the benign tumor doesn’t have fatal harm for you . [SEP]
[CLS] the malignant tumor doesn’t have fatal harm for you . [SEP]
[CLS] kids catch butterflies for food . [SEP]
[CLS] kids catch butterflies for fun . [SEP]
[CLS] drinking milk can help teenagers grow taller . [SEP]
[CLS] drinking milk can help teenagers grow shorter . [SEP]
[CLS] I used to take notes on the pens . [SEP]
[CLS] I used to take notes on the paper . [SEP]
[CLS] I use a knife to eat rice . [SEP]
[CLS] I use a spoon to eat rice . [SEP]
[CLS] students need to write their scores on the examination paper . [SEP]
[CLS] students need to write their names on the examination paper . [SEP]
[CLS] perming will make your hair curly . [SEP]
[CLS] perming will make your hair longer . [SEP]
[CLS] twi

[CLS] people expect enjoyment when they go to the cinema . [SEP]
[CLS] people expect boredom when they go to the cinema . [SEP]
[CLS] Bob sang a picture yesterday . [SEP]
[CLS] Bob painted a picture yesterday . [SEP]
[CLS] paying by credit card is for not paying money . [SEP]
[CLS] paying by credit card is for not carrying cash . [SEP]
[CLS] you need an audience if you want to share your story . [SEP]
[CLS] you need a dog if you want to share your story . [SEP]
[CLS] the players will buy a cup of beer before the basketball game begins . [SEP]
[CLS] the audience will buy a cup of beer before the basketball game begins . [SEP]
[CLS] I need to say goodbye to people on TV when I finish watching the TV show . [SEP]
[CLS] I need to turn off the TV when I finish watching the TV show . [SEP]
[CLS] goods can be transported by plane . [SEP]
[CLS] goods can be transported by wind . [SEP]
[CLS] the ship hit the submarine yesterday . [SEP]
[CLS] the ship hit the train yesterday . [SEP]
[CLS] I can 

[CLS] Something you might do while taking the bus is drawing an oil painting . [SEP]
[CLS] Something you might do while taking the bus is reading a newspaper . [SEP]
[CLS] She felt full and satisfied after having lunch . [SEP]
[CLS] She felt hungry and satisfied after having lunch . [SEP]
[CLS] Bob is cooking while eating dinner . [SEP]
[CLS] Bob is chewing while eating dinner . [SEP]
[CLS] A rat can survive a fall from a five-floor building . [SEP]
[CLS] A person can survive a fall from a five-floor building . [SEP]
[CLS] The longest nonstop flight by a bird is eight days . [SEP]
[CLS] The longest nonstop flight by a dog is eight days . [SEP]
[CLS] He caught a cold and had a hot shower . [SEP]
[CLS] He caught a cold and had a cold shower . [SEP]
[CLS] Humans smell through their ears . [SEP]
[CLS] Humans smell through their noses . [SEP]
[CLS] Alice formed a habit of sleeping to keep fit . [SEP]
[CLS] Alice formed a habit of jogging to keep fit . [SEP]
[CLS] It's winter and it's freezi

[CLS] minaj was deemed mentally ill so she sought a career as psychologist . [SEP]
[CLS] minaj was deemed mentally ill so she was sent to a psychologist . [SEP]
[CLS] the aggressive football coach lost his voice after the game . [SEP]
[CLS] the aggressive football coach lost his voice before the game . [SEP]
[CLS] I applied pressure to the cut on my arm and it healed . [SEP]
[CLS] I applied pressure to the cut on my arm and it stopped bleeding . [SEP]
[CLS] leo had an infection so she washed her hands . [SEP]
[CLS] leo had an infection so she took antibiotic . [SEP]
[CLS] the woman fanned herself with her hand for the air conditioner is new . [SEP]
[CLS] the woman fanned herself with her hand for the air conditioner is broken . [SEP]
[CLS] the administrator cleared her throat because the meeting was commenced . [SEP]
[CLS] the administrator cleared her throat because the meeting was postponed . [SEP]
[CLS] the boy giggled uncontrollably because his brother kicked him . [SEP]
[CLS] the 

In [21]:
def get_results_from_perps(perps):
    results = []
    for i in range(0, len(perps), 2):
        if perps[i] < perps[i + 1]:
            results.append(1)
        else:
            results.append(0)
    return results

In [72]:
results

[1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,


In [25]:
tuples = []
for i in range(0, len(perps), 2):
    tuples.append((perps[i], perps[i + 1]))

In [30]:
tuples[:10]

[(363.13889945607536, 379.4795891155589),
 (56.93333855729592, 57.27293636965309),
 (104.42259540038052, 299.1831727083033),
 (97.81770213797813, 31.509369872993396),
 (22.22480847168275, 28.20688877106866),
 (53.811270341874895, 56.527437677318275),
 (70.54278372179634, 63.33405605900588),
 (2180.012970087111, 1567.7849056429868),
 (214.25113953713992, 179.53446980499257),
 (1075.618261046155, 1246.4587021811444)]

In [54]:
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /Users/goelprat/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
INFO:transformers.configuration_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

INFO:transformers.modeling_utils:loading weights file https://s3.amazona

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [63]:
def run(i):
    tokenized_text = tokenizer.tokenize(testdf.sent[i])
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    o = model(tokens_tensor)
    loss = loss_fct(o[0].squeeze(),tokens_tensor.squeeze()).data 
    return math.exp(loss)

In [65]:
run(6)

3.791765494752476

In [32]:
tokenizer.tokenize(testdf.sent[1])

['he', 'put', 'a', 'turkey', 'into', 'the', 'fridge']

In [18]:
results

[1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,


In [ ]:
##################### Using GPT-2 to get sentence probability #########################

In [109]:
import math
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
# Load pre-trained model (weights)
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')
gpt_model.eval()
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

loss_fct = torch.nn.CrossEntropyLoss()



In [179]:
def gpt_score(sentence):
    sentence = ' <|endoftext|> ' + sentence
    tokenize_input = tokenizer.tokenize(sentence, add_prefix_space=True)
    print(tokenize_input)
    tokens_tensor = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions=gpt_model(tokens_tensor)
    loss = loss_fct(predictions[0].squeeze()[:-1],tokens_tensor.squeeze()[1:]).data 
    return math.exp(loss)

def gpt_tokens(input_sentence):
    sentence = '<|endoftext|> ' + input_sentence
    tokenize_input = tokenizer.tokenize(sentence)
    tokens_tensor = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    predictions =gpt_model(tokens_tensor)
    predicted_tokens = []
    for i in range(tokens_tensor.shape[1]):
        predicted_index = torch.argmax(predictions[0][0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
        predicted_tokens.append(predicted_token)
    return predicted_tokens

In [111]:
def run(sents, mode='perp'):
    out = []
    with torch.no_grad():
        for sent in sents:
            if mode == 'perp':
                out.append(gpt_score(sent))
            else:
                out.append(gpt_tokens(sent))
    return out

In [182]:
run(['Computers put my desktop into my suitcase', 'I put my laptop into my suitcase'], mode='perp')

['<|endoftext|>', 'ĠComput', 'ers', 'Ġput', 'Ġmy', 'Ġdesktop', 'Ġinto', 'Ġmy', 'Ġsuitcase']
['<|endoftext|>', 'ĠI', 'Ġput', 'Ġmy', 'Ġlaptop', 'Ġinto', 'Ġmy', 'Ġsuitcase']


[485.89971481285045, 121.18085829248955]

In [123]:
sents = [x for x in testdf.sent]

In [124]:
perps = run(sents)

['<|endoftext|>', 'Ġhe', 'Ġput', 'Ġan', 'Ġelephant', 'Ġinto', 'Ġthe', 'Ġfridge']
['<|endoftext|>', 'Ġhe', 'Ġput', 'Ġa', 'Ġturkey', 'Ġinto', 'Ġthe', 'Ġfridge']
['<|endoftext|>', 'Ġmy', 'Ġsister', 'Ġeats', 'Ġan', 'Ġapple', 'Ġafter', 'Ġbreakfast', 'Ġevery', 'Ġday']
['<|endoftext|>', 'Ġmy', 'Ġsister', 'Ġeats', 'Ġa', 'Ġstone', 'Ġafter', 'Ġbreakfast', 'Ġevery', 'Ġday']
['<|endoftext|>', 'Ġmoney', 'Ġcan', 'Ġbe', 'Ġused', 'Ġfor', 'Ġbuying', 'Ġcars']
['<|endoftext|>', 'Ġmoney', 'Ġcan', 'Ġbe', 'Ġused', 'Ġfor', 'Ġbuying', 'Ġstars']
['<|endoftext|>', 'ĠNew', 'ĠYork', 'Ġis', 'Ġlocated', 'Ġin', 'Ġthe', 'Ġnortheastern', 'Ġpart', 'Ġof', 'ĠUSA']
['<|endoftext|>', 'ĠUSA', 'Ġis', 'Ġlocated', 'Ġin', 'Ġthe', 'Ġnortheastern', 'Ġpart', 'Ġof', 'ĠNew', 'ĠYork']
['<|endoftext|>', 'Ġa', 'Ġman', 'Ġcan', 'Ġbetter', 'Ġsee', 'Ġstars', 'Ġand', 'Ġthe', 'Ġmoon', 'Ġin', 'Ġdaytime']
['<|endoftext|>', 'Ġa', 'Ġman', 'Ġcan', 'Ġhardly', 'Ġsee', 'Ġstars', 'Ġand', 'Ġthe', 'Ġmoon', 'Ġin', 'Ġdaytime']
['<|endoftext|>', 'Ġhe', 'Ġ

['<|endoftext|>', 'Ġwool', 'Ġis', 'Ġa', 'Ġform', 'Ġof', 'Ġclothing', 'Ġmaterial']
['<|endoftext|>', 'Ġwool', 'Ġis', 'Ġa', 'Ġform', 'Ġof', 'Ġeating', 'Ġmaterial']
['<|endoftext|>', 'Ġmum', 'Ġis', 'Ġcooking', 'Ġa', 'Ġdelicious', 'Ġmeal', 'Ġin', 'Ġthe', 'Ġkitchen']
['<|endoftext|>', 'Ġmum', 'Ġis', 'Ġcooking', 'Ġa', 'Ġdelicious', 'Ġmeal', 'Ġin', 'Ġthe', 'Ġbedroom']
['<|endoftext|>', 'Ġcars', 'Ġon', 'Ġthe', 'Ġfreeway', 'Ġmultiply', 'Ġduring', 'Ġholidays']
['<|endoftext|>', 'Ġcars', 'Ġon', 'Ġthe', 'Ġrailway', 'Ġmultiply', 'Ġduring', 'Ġholidays']
['<|endoftext|>', 'Ġbeef', 'Ġcooked', 'Ġon', 'Ġthe', 'Ġdrill', 'Ġtastes', 'Ġdelicious']
['<|endoftext|>', 'Ġbeef', 'Ġcooked', 'Ġon', 'Ġthe', 'Ġgrill', 'Ġtastes', 'Ġdelicious']
['<|endoftext|>', 'ĠJerry', 'Ġtakes', 'Ġa', 'Ġvisiting', 'Ġcard', 'Ġfrom', 'Ġthe', 'Ġro', 'l', 'od', 'ex', 'Ġto', 'Ġkeep', 'Ġin', 'Ġtouch', 'Ġwith', 'ĠTom']
['<|endoftext|>', 'ĠJerry', 'Ġtakes', 'Ġa', 'Ġtissue', 'Ġfrom', 'Ġthe', 'Ġro', 'l', 'od', 'ex', 'Ġto', 'Ġkeep', 'Ġin', 'Ġ

['<|endoftext|>', 'Ġi', 'Ġfly', 'Ġto', 'ĠParis', 'Ġto', 'Ġsee', 'Ġthe', 'ĠStatue', 'Ġof', 'ĠLiberty']
['<|endoftext|>', 'Ġn', 'issan', 'Ġis', 'Ġa', 'Ġforeign', 'Ġplane', 'Ġmanufacturer']
['<|endoftext|>', 'Ġn', 'issan', 'Ġis', 'Ġa', 'Ġforeign', 'Ġcar', 'Ġmanufacturer']
['<|endoftext|>', 'Ġfood', 'Ġis', 'Ġstored', 'Ġin', 'Ġthe', 'Ġpan']
['<|endoftext|>', 'Ġfood', 'Ġis', 'Ġstored', 'Ġin', 'Ġthe', 'Ġfridge']
['<|endoftext|>', 'Ġthe', 'Ġimage', 'Ġof', 'Ġthe', 'Ġcat', 'Ġseems', 'Ġto', 'Ġbe', 'Ġreflected', 'Ġmany', 'Ġtimes', 'Ġin', 'Ġthe', 'Ġmirror']
['<|endoftext|>', 'Ġthe', 'Ġimage', 'Ġof', 'Ġthe', 'Ġcat', 'Ġseems', 'Ġto', 'Ġbe', 'Ġreflected', 'Ġmany', 'Ġtimes', 'Ġin', 'Ġthe', 'Ġglass']
['<|endoftext|>', 'Ġshe', 'Ġgoes', 'Ġtraveling', 'Ġon', 'Ġholidays']
['<|endoftext|>', 'Ġshe', 'Ġgoes', 'Ġtraveling', 'Ġon', 'Ġweek', 'days']
['<|endoftext|>', 'Ġlaughter', 'Ġis', 'Ġa', 'Ġsign', 'Ġof', 'Ġjoy']
['<|endoftext|>', 'Ġlaughter', 'Ġis', 'Ġa', 'Ġsign', 'Ġof', 'Ġsorrow']
['<|endoftext|>', 'ĠMary', 

['<|endoftext|>', 'Ġpeople', 'Ġmust', 'Ġhave', 'Ġa', 'Ġpassport', 'Ġto', 'Ġtravel', 'Ġabroad']
['<|endoftext|>', 'Ġpeople', 'Ġmust', 'Ġhave', 'Ġa', 'Ġdriver', "'s", 'Ġlicense', 'Ġto', 'Ġtravel', 'Ġabroad']
['<|endoftext|>', 'Ġit', 'Ġis', 'Ġsnowy', 'Ġand', 'Ġvery', 'Ġhot', 'Ġoutside']
['<|endoftext|>', 'Ġit', 'Ġis', 'Ġsnowy', 'Ġand', 'Ġvery', 'Ġcold', 'Ġoutside']
['<|endoftext|>', 'Ġthe', 'Ġcar', 'Ġstops', 'Ġat', 'Ġthe', 'Ġgreen', 'Ġlight']
['<|endoftext|>', 'Ġthe', 'Ġcar', 'Ġstops', 'Ġat', 'Ġthe', 'Ġred', 'Ġlight']
['<|endoftext|>', 'Ġpeople', 'Ġput', 'Ġon', 'Ġclothes', 'Ġwhen', 'Ġthey', 'Ġfeel', 'Ġcold']
['<|endoftext|>', 'Ġpeople', 'Ġtake', 'Ġoff', 'Ġclothes', 'Ġwhen', 'Ġthey', 'Ġfeel', 'Ġcold']
['<|endoftext|>', 'ĠTrump', 'Ġwas', 'ĠEmperor', 'Ġof', 'Ġthe', 'ĠUSA', 'Ġin', 'Ġ2018']
['<|endoftext|>', 'ĠTrump', 'Ġwas', 'ĠPresident', 'Ġof', 'Ġthe', 'ĠUSA', 'Ġin', 'Ġ2018']
['<|endoftext|>', 'Ġhe', 'Ġputs', 'Ġthe', 'Ġradio', 'Ġnext', 'Ġto', 'Ġhis', 'Ġmouth', 'Ġto', 'Ġhear', 'Ġmore', 'Ġclea

['<|endoftext|>', 'Ġi', 'Ġam', 'Ġpreparing', 'Ġfor', 'Ġthe', 'Ġtrip', 'Ġyesterday']
['<|endoftext|>', 'Ġi', 'Ġbegin', 'Ġwalking', 'Ġas', 'Ġsoon', 'Ġas', 'Ġthe', 'Ġlight', 'Ġturns', 'Ġgreen']
['<|endoftext|>', 'Ġi', 'Ġbegin', 'Ġwalking', 'Ġas', 'Ġsoon', 'Ġas', 'Ġthe', 'Ġlight', 'Ġturns', 'Ġpurple']
['<|endoftext|>', 'Ġi', 'Ġdrink', 'Ġco', 'ke', 'Ġwith', 'Ġa', 'Ġchop', 'stick']
['<|endoftext|>', 'Ġi', 'Ġdrink', 'Ġco', 'ke', 'Ġwith', 'Ġa', 'Ġstraw']
['<|endoftext|>', 'Ġthe', 'Ġcat', 'Ġenjoys', 'Ġplaying', 'Ġknitting', 'Ġwool']
['<|endoftext|>', 'Ġthe', 'Ġknitting', 'Ġwool', 'Ġenjoys', 'Ġplaying', 'Ġcats']
['<|endoftext|>', 'Ġshe', 'Ġtakes', 'Ġattractive', 'Ġselfies', 'Ġwith', 'Ġher', 'Ġsmartphone']
['<|endoftext|>', 'Ġshe', 'Ġtakes', 'Ġattractive', 'Ġselfies', 'Ġwith', 'Ġher', 'Ġheadphone']
['<|endoftext|>', 'Ġi', 'Ġbuy', 'Ġa', 'Ġboyfriend', 'Ġfor', 'Ġmy', 'Ġsister']
['<|endoftext|>', 'Ġi', 'Ġbuy', 'Ġa', 'Ġring', 'Ġfor', 'Ġmy', 'Ġsister']
['<|endoftext|>', 'ĠHis', 'Ġfather', 'Ġgot', 'Ġang

...

['<|endoftext|>', 'Ġadults', 'Ġchange', 'Ġclothes', 'Ġaccording', 'Ġto', 'Ġtheir', 'Ġheight', 'Ġevery', 'Ġday']
['<|endoftext|>', 'Ġyou', 'Ġcan', 'Ġuse', 'Ġink', 'Ġto', 'Ġwrite', 'Ġthings']
['<|endoftext|>', 'Ġyou', 'Ġcan', 'Ġuse', 'Ġa', 'Ġlink', 'Ġto', 'Ġwrite', 'Ġthings']
['<|endoftext|>', 'Ġmuseum', 'Ġexhibits', 'Ġartifacts', 'Ġfor', 'Ġeveryone', 'Ġto', 'Ġlearn']
['<|endoftext|>', 'Ġmuseum', 'Ġexhibits', 'Ġartifacts', 'Ġfor', 'Ġsale']
['<|endoftext|>', 'Ġthe', 'Ġalarm', 'Ġringing', 'Ġannounced', 'Ġthat', 'Ġthe', 'Ġclass', 'Ġwas', 'Ġover']
['<|endoftext|>', 'Ġthe', 'Ġbell', 'Ġringing', 'Ġannounced', 'Ġthat', 'Ġthe', 'Ġclass', 'Ġwas', 'Ġover']
['<|endoftext|>', 'Ġpassengers', 'Ġneed', 'Ġto', 'Ġundergo', 'Ġa', 'Ġrigorous', 'Ġsafety', 'Ġcheck', 'Ġbefore', 'Ġboarding', 'Ġthe', 'Ġplane']
['<|endoftext|>', 'Ġpassengers', 'Ġneed', 'Ġto', 'Ġundergo', 'Ġa', 'Ġrigorous', 'Ġsafety', 'Ġcheck', 'Ġbefore', 'Ġtaking', 'Ġa', 'Ġtaxi']
['<|endoftext|>', 'Ġyou', 'Ġcan', 'Ġuse', 'Ġmoney', 'Ġto', 'Ġbuy',

['<|endoftext|>', 'ĠI', 'Ġshould', 'Ġrefresh', 'Ġmyself', 'Ġwhen', 'ĠI', 'Ġfail']
['<|endoftext|>', 'Ġmom', 'Ġcooks', 'Ġthe', 'Ġfried', 'Ġegg', 'Ġonce', 'Ġa', 'Ġweek', 'Ġfor', 'Ġme']
['<|endoftext|>', 'Ġmom', 'Ġcooks', 'Ġthe', 'Ġfried', 'Ġtiger', 'Ġmeat', 'Ġonce', 'Ġa', 'Ġweek', 'Ġfor', 'Ġme']
['<|endoftext|>', 'Ġspr', 'inters', "'", 'Ġlegs', 'Ġare', 'Ġusually', 'Ġstrong']
['<|endoftext|>', 'Ġspr', 'inters', "'", 'Ġarms', 'Ġare', 'Ġusually', 'Ġstrong']
['<|endoftext|>', 'ĠI', 'Ġcan', 'Ġtake', 'Ġpart', 'Ġin', 'Ġthe', 'Ġsoccer', 'Ġgame', 'Ġin', 'Ġformal', 'Ġclothes']
['<|endoftext|>', 'ĠI', 'Ġcan', 'Ġtake', 'Ġpart', 'Ġin', 'Ġthe', 'Ġsoccer', 'Ġgame', 'Ġin', 'Ġathletic', 'Ġclothes']
['<|endoftext|>', 'Ġwe', 'Ġcan', 'Ġtravel', 'Ġon', 'Ġthe', 'Ġsea', 'Ġby', 'Ġboat']
['<|endoftext|>', 'Ġwe', 'Ġcan', 'Ġtravel', 'Ġon', 'Ġthe', 'Ġsea', 'Ġby', 'Ġtrain']
['<|endoftext|>', 'Ġchickens', "'", 'Ġeggs', 'Ġis', 'Ġdelicious']
['<|endoftext|>', 'Ġpigs', "'", 'Ġeggs', 'Ġare', 'Ġdelicious']
['<|endoftext|>

['<|endoftext|>', 'Ġair', 'Ġleaked', 'Ġout', 'Ġof', 'Ġthe', 'Ġbeach', 'Ġball', 'Ġbecause', 'Ġit', 'Ġhad', 'Ġa', 'Ġpicture', 'Ġon', 'Ġit']
['<|endoftext|>', 'Ġair', 'Ġleaked', 'Ġout', 'Ġof', 'Ġthe', 'Ġbeach', 'Ġball', 'Ġbecause', 'Ġit', 'Ġhad', 'Ġa', 'Ġhole', 'Ġon', 'Ġit']
['<|endoftext|>', 'Ġthe', 'Ġconversation', 'Ġgradually', 'Ġlull', 'ed', 'Ġto', 'Ġsilence', 'Ġas', 'Ġmore', 'Ġpeople', 'Ġleft']
['<|endoftext|>', 'Ġthe', 'Ġconversation', 'Ġgradually', 'Ġlull', 'ed', 'Ġto', 'Ġsilence', 'Ġas', 'Ġmore', 'Ġpeople', 'Ġjoined']
['<|endoftext|>', 'Ġj', 'ulia', 'Ġpressed', 'Ġdown', 'Ġon', 'Ġa', 'Ġpiano', 'Ġkey', 'Ġto', 'Ġplay', 'Ġa', 'Ġmusic']
['<|endoftext|>', 'Ġj', 'ulia', 'Ġpressed', 'Ġdown', 'Ġon', 'Ġa', 'Ġpiano', 'Ġkey', 'Ġto', 'Ġmake', 'Ġa', 'Ġsound']
['<|endoftext|>', 'Ġvehicles', 'Ġtravelled', 'Ġslowly', 'Ġin', 'Ġcloudy', 'Ġdays']
['<|endoftext|>', 'Ġvehicles', 'Ġtravelled', 'Ġslowly', 'Ġin', 'Ġfog', 'gy', 'Ġdays']
['<|endoftext|>', 'Ġla', 'uren', 'Ġgained', 'Ġweight', 'Ġso', 'Ġshe', 

['<|endoftext|>', 'Ġle', 'o', 'Ġhad', 'Ġan', 'Ġinfection', 'Ġso', 'Ġshe', 'Ġtook', 'Ġantibiotic']
['<|endoftext|>', 'Ġthe', 'Ġwoman', 'Ġf', 'anned', 'Ġherself', 'Ġwith', 'Ġher', 'Ġhand', 'Ġfor', 'Ġthe', 'Ġair', 'Ġcondition', 'er', 'Ġis', 'Ġnew']
['<|endoftext|>', 'Ġthe', 'Ġwoman', 'Ġf', 'anned', 'Ġherself', 'Ġwith', 'Ġher', 'Ġhand', 'Ġfor', 'Ġthe', 'Ġair', 'Ġcondition', 'er', 'Ġis', 'Ġbroken']
['<|endoftext|>', 'Ġthe', 'Ġadministrator', 'Ġcleared', 'Ġher', 'Ġthroat', 'Ġbecause', 'Ġthe', 'Ġmeeting', 'Ġwas', 'Ġcommenced']
['<|endoftext|>', 'Ġthe', 'Ġadministrator', 'Ġcleared', 'Ġher', 'Ġthroat', 'Ġbecause', 'Ġthe', 'Ġmeeting', 'Ġwas', 'Ġpostponed']
['<|endoftext|>', 'Ġthe', 'Ġboy', 'Ġgigg', 'led', 'Ġuncontroll', 'ably', 'Ġbecause', 'Ġhis', 'Ġbrother', 'Ġkicked', 'Ġhim']
['<|endoftext|>', 'Ġthe', 'Ġboy', 'Ġgigg', 'led', 'Ġuncontroll', 'ably', 'Ġbecause', 'Ġhis', 'Ġbrother', 'Ġtick', 'led', 'Ġhim']
['<|endoftext|>', 'Ġthe', 'ĠDJ', 'Ġturned', 'Ġthe', 'Ġmusic', 'Ġon', 'Ġand', 'Ġeveryone', 'Ġ

['<|endoftext|>', 'Ġi', 'Ġpick', 'Ġa', 'Ġleaf', 'Ġfrom', 'Ġa', 'Ġmaple', 'Ġtree']
['<|endoftext|>', 'Ġi', 'Ġpick', 'Ġa', 'Ġleaf', 'Ġfrom', 'Ġa', 'Ġgold', 'fish']
['<|endoftext|>', 'Ġi', 'Ġam', 'Ġlearning', 'ĠFrench', 'Ġfrom', 'Ġmy', 'Ġteacher']
['<|endoftext|>', 'Ġi', 'Ġam', 'Ġlearning', 'ĠFrench', 'Ġfrom', 'Ġmy', 'Ġcat']
['<|endoftext|>', 'ĠIndia', 'Ġis', 'Ġa', 'Ġcountry', 'Ġin', 'ĠEurope']
['<|endoftext|>', 'ĠGreat', 'ĠBritain', 'Ġis', 'Ġa', 'Ġcountry', 'Ġin', 'ĠEurope']
['<|endoftext|>', 'Ġyou', 'Ġcan', 'Ġride', 'Ġfrom', 'ĠAmerica', 'Ġto', 'ĠJapan']
['<|endoftext|>', 'Ġyou', 'Ġcan', 'Ġfly', 'Ġfrom', 'ĠAmerica', 'Ġto', 'ĠJapan']
['<|endoftext|>', 'ĠBob', 'Ġgoes', 'Ġto', 'Ġbed', 'Ġbecause', 'Ġhe', 'Ġfeels', 'Ġsleepy']
['<|endoftext|>', 'ĠBob', 'Ġgoes', 'Ġto', 'Ġwork', 'Ġbecause', 'Ġhe', 'Ġfeels', 'Ġsleepy']
['<|endoftext|>', 'ĠEllen', 'Ġkicks', 'Ġthe', 'Ġjudge', 'Ġhard', 'Ġin', 'Ġorder', 'Ġto', 'Ġwin']
['<|endoftext|>', 'ĠEllen', 'Ġkicks', 'Ġthe', 'Ġball', 'Ġhard', 'Ġin', 'Ġorder', 'Ġ

In [125]:
def get_results_from_perps(perps):
    results = []
    for i in range(0, len(perps), 2):
        if perps[i] < perps[i + 1]:
            results.append(1)
        else:
            results.append(0)
    return results

In [122]:
for s, p in zip(testdf[:10].sent, perps):
    print(p, s)

267.39896106198194 he put an elephant into the fridge
208.89728072559373 he put a turkey into the fridge
211.21632855786632 my sister eats an apple after breakfast every day
393.27129919939523 my sister eats a stone after breakfast every day
133.9082425146872 money can be used for buying cars
345.7681458323154 money can be used for buying stars
67.60459646620482 New York is located in the northeastern part of USA
53.93842060274841 USA is located in the northeastern part of New York
336.4607314720642 a man can better see stars and the moon in daytime
252.25760840082438 a man can hardly see stars and the moon in daytime


In [126]:
results = get_results_from_perps(perps)

In [87]:
results[:10]

[1, 1, 0, 1, 1, 1, 0, 0, 1, 1]

In [130]:
test_answers = pd.read_csv('data/trial/taskA_trial_answer.csv', header=None)

In [132]:
test_answers[1]

0       0
1       1
2       1
3       1
4       0
       ..
2016    1
2017    1
2018    1
2019    0
2020    0
Name: 1, Length: 2021, dtype: int64

In [143]:
wrong_examples = x_test[test_answers[1] != pd.Series(results)]

In [160]:
wrong_examples

,id,sent0,sent1
3,4,New York is located in the northeastern part o...,USA is located in the northeastern part of New...
6,7,his mother became satisfied after he got good ...,his mother became angry after he got good grad...
13,14,this man can jump across a mountain easily,this man can jump across a hurdle easily
22,23,I changed my direction when passing a crossroads,I changed my car when passing a crossroads
26,27,Bob played hard to prepare for the examination,Bob studied hard to prepare for the examination
...,...,...,...
1995,1996,she prepares a custom-designed nightgown for t...,she prepares a custom-designed evening dress f...
2002,2003,he is drawing a mountain on a piece of paper,he is climbing a mountain on a piece of paper
2005,2006,we have lunch in the restaurant,we have lunch in the library
2011,2012,India is a country in Europe,Great Britain is a country in Europe


In [ ]:
perps = run(sents)

In [162]:
wrong_examples

,id,sent0,sent1
3,4,New York is located in the northeastern part o...,USA is located in the northeastern part of New...
6,7,his mother became satisfied after he got good ...,his mother became angry after he got good grad...
13,14,this man can jump across a mountain easily,this man can jump across a hurdle easily
22,23,I changed my direction when passing a crossroads,I changed my car when passing a crossroads
26,27,Bob played hard to prepare for the examination,Bob studied hard to prepare for the examination
...,...,...,...
1995,1996,she prepares a custom-designed nightgown for t...,she prepares a custom-designed evening dress f...
2002,2003,he is drawing a mountain on a piece of paper,he is climbing a mountain on a piece of paper
2005,2006,we have lunch in the restaurant,we have lunch in the library
2011,2012,India is a country in Europe,Great Britain is a country in Europe


In [173]:
items = []
for row in wrong_examples.itertuples():
    items.append({'id' : row.id, 'sent' : row.sent0})
    items.append({'id' : row.id, 'sent' : row.sent1})
wrongdf = pd.DataFrame(items)

In [174]:
perps = run(wrongdf.sent)

['<|endoftext|>', 'ĠNew', 'ĠYork', 'Ġis', 'Ġlocated', 'Ġin', 'Ġthe', 'Ġnortheastern', 'Ġpart', 'Ġof', 'ĠUSA']
['<|endoftext|>', 'ĠUSA', 'Ġis', 'Ġlocated', 'Ġin', 'Ġthe', 'Ġnortheastern', 'Ġpart', 'Ġof', 'ĠNew', 'ĠYork']
['<|endoftext|>', 'Ġhis', 'Ġmother', 'Ġbecame', 'Ġsatisfied', 'Ġafter', 'Ġhe', 'Ġgot', 'Ġgood', 'Ġgrades', 'Ġin', 'Ġthe', 'Ġmath', 'Ġexam']
['<|endoftext|>', 'Ġhis', 'Ġmother', 'Ġbecame', 'Ġangry', 'Ġafter', 'Ġhe', 'Ġgot', 'Ġgood', 'Ġgrades', 'Ġin', 'Ġthe', 'Ġmath', 'Ġexam']
['<|endoftext|>', 'Ġthis', 'Ġman', 'Ġcan', 'Ġjump', 'Ġacross', 'Ġa', 'Ġmountain', 'Ġeasily']
['<|endoftext|>', 'Ġthis', 'Ġman', 'Ġcan', 'Ġjump', 'Ġacross', 'Ġa', 'Ġhurdle', 'Ġeasily']
['<|endoftext|>', 'ĠI', 'Ġchanged', 'Ġmy', 'Ġdirection', 'Ġwhen', 'Ġpassing', 'Ġa', 'Ġcross', 'roads']
['<|endoftext|>', 'ĠI', 'Ġchanged', 'Ġmy', 'Ġcar', 'Ġwhen', 'Ġpassing', 'Ġa', 'Ġcross', 'roads']
['<|endoftext|>', 'ĠBob', 'Ġplayed', 'Ġhard', 'Ġto', 'Ġprepare', 'Ġfor', 'Ġthe', 'Ġexamination']
['<|endoftext|>', 'ĠBob

['<|endoftext|>', 'Ġpump', 'kins', 'Ġcan', 'Ġbe', 'Ġmade', 'Ġinto', 'Ġcar', 'riages', 'Ġfor', 'Ġpeople', 'Ġto', 'Ġride', 'Ġon']
['<|endoftext|>', 'Ġi', 'Ġuse', 'Ġscissors', 'Ġto', 'Ġcut', 'Ġpaper', 'Ġcuts']
['<|endoftext|>', 'Ġi', 'Ġuse', 'Ġpencil', 's', 'Ġto', 'Ġcut', 'Ġpaper', 'Ġcuts']
['<|endoftext|>', 'ĠTony', 'Ġgives', 'Ġhis', 'Ġgirlfriend', 'Ġch', 'ocol', 'ates', 'Ġto', 'Ġpropose']
['<|endoftext|>', 'ĠTony', 'Ġgives', 'Ġhis', 'Ġgirlfriend', 'Ġch', 'ocol', 'ates', 'Ġto', 'Ġbreak', 'Ġup']
['<|endoftext|>', 'Ġthe', 'Ġdog', 'Ġsits', 'Ġthere', 'Ġwithout', 'Ġeven', 'Ġa', 'Ġwo', 'of']
['<|endoftext|>', 'Ġthe', 'Ġdog', 'Ġsits', 'Ġthere', 'Ġwithout', 'Ġeven', 'Ġa', 'Ġm', 'ew']
['<|endoftext|>', 'Ġhe', 'Ġwrites', 'Ġhis', 'Ġhomework', 'Ġon', 'Ġthe', 'Ġde', 'w']
['<|endoftext|>', 'Ġhe', 'Ġwrites', 'Ġhis', 'Ġhomework', 'Ġon', 'Ġthe', 'Ġnotebook']
['<|endoftext|>', 'Ġa', 'Ġrabbit', 'Ġlikes', 'Ġeating', 'Ġmeat']
['<|endoftext|>', 'Ġa', 'Ġrabbit', 'Ġlikes', 'Ġeating', 'Ġcarrots']
['<|endoftext|>

['<|endoftext|>', 'Ġmy', 'Ġfriends', 'Ġplay', 'Ġsoccer', 'Ġball', 'Ġin', 'Ġthe', 'Ġgym', 'nas', 'ium']
['<|endoftext|>', 'Ġpeople', 'Ġfeel', 'Ġpleasure', 'Ġwhile', 'Ġkissing']
['<|endoftext|>', 'Ġpeople', 'Ġfeel', 'Ġsick', 'Ġwhile', 'Ġkissing']
['<|endoftext|>', 'Ġpeople', 'Ġusually', 'Ġeat', 'Ġice', 'Ġcream', 'Ġin', 'Ġsummer']
['<|endoftext|>', 'Ġpeople', 'Ġusually', 'Ġeat', 'Ġice', 'Ġcream', 'Ġin', 'Ġwinter']
['<|endoftext|>', 'ĠI', 'Ġsaw', 'Ġa', 'Ġround', 'Ġpizza', '.']
['<|endoftext|>', 'ĠI', 'Ġsaw', 'Ġa', 'Ġsquare', 'Ġpizza', '.']
['<|endoftext|>', 'ĠI', 'Ġhave', 'Ġfree', 'Ġaccess', 'Ġto', 'Ġsome', 'Ġmuseums', 'Ġwith', 'Ġtickets']
['<|endoftext|>', 'ĠI', 'Ġhave', 'Ġfree', 'Ġaccess', 'Ġto', 'Ġall', 'Ġmuseums', 'Ġwithout', 'Ġtickets']
['<|endoftext|>', 'ĠWe', 'Ġcut', 'Ġdown', 'Ġtrees', 'Ġto', 'Ġprotect', 'Ġthe', 'Ġenvironment']
['<|endoftext|>', 'ĠWe', 'Ġplant', 'Ġtrees', 'Ġto', 'Ġprotect', 'Ġthe', 'Ġenvironment']
['<|endoftext|>', 'ĠMy', 'Ġbrother', 'Ġis', 'Ġmy', 'Ġfamily', 'Ġmembe

['<|endoftext|>', 'Ġthe', 'Ġbag', 'Ġis', 'Ġtoo', 'Ġweight', 'y', ',', 'Ġso', 'ĠI', 'Ġcannot', 'Ġhold', 'Ġit']
['<|endoftext|>', 'Ġthe', 'Ġbag', 'Ġis', 'Ġtoo', 'Ġlight', ',', 'Ġso', 'ĠI', 'Ġcannot', 'Ġhold', 'Ġit']
['<|endoftext|>', 'ĠI', 'Ġfinished', 'Ġmy', 'Ġwork', 'Ġlate', 'Ġlast', 'Ġnight', 'Ġso', 'ĠI', 'Ġam', 'Ġenergetic', 'Ġnow']
['<|endoftext|>', 'ĠI', 'Ġfinished', 'Ġmy', 'Ġwork', 'Ġearly', 'Ġlast', 'Ġnight', 'Ġso', 'ĠI', 'Ġam', 'Ġenergetic', 'Ġnow']
['<|endoftext|>', 'ĠI', 'Ġget', 'Ġup', 'Ġtoo', 'Ġearly', 'Ġso', 'ĠI', 'Ġwill', 'Ġbe', 'Ġlate', 'Ġfor', 'Ġschool']
['<|endoftext|>', 'ĠI', 'Ġget', 'Ġup', 'Ġtoo', 'Ġlate', 'Ġso', 'ĠI', 'Ġwill', 'Ġbe', 'Ġlate', 'Ġfor', 'Ġschool']
['<|endoftext|>', 'Ġhe', 'Ġuses', 'Ġa', 'Ġbamboo', 'Ġbasket', 'Ġto', 'Ġfetch', 'Ġwater']
['<|endoftext|>', 'Ġhe', 'Ġuses', 'Ġa', 'Ġbottle', 'Ġto', 'Ġfetch', 'Ġwater']
['<|endoftext|>', 'Ġthey', 'Ġfeel', 'Ġcolder', 'Ġafter', 'Ġthe', 'Ġsnow']
['<|endoftext|>', 'ĠThey', 'Ġfeel', 'Ġwarmer', 'Ġafter', 'Ġthe', 'Ġsnow

['<|endoftext|>', 'ĠIf', 'Ġpeople', 'Ġhave', 'Ġsomething', 'Ġurgent', 'Ġto', 'Ġgo', 'Ġto', 'Ġa', 'Ġplace', ',', 'Ġthey', 'Ġusually', 'Ġwalk']
['<|endoftext|>', 'ĠIf', 'Ġpeople', 'Ġhave', 'Ġsomething', 'Ġurgent', 'Ġto', 'Ġgo', 'Ġto', 'Ġa', 'Ġplace', 'Ġthey', 'Ġusually', 'Ġtake', 'Ġa', 'Ġplane']
['<|endoftext|>', 'ĠMost', 'Ġpeople', 'Ġdon', "'t", 'Ġneed', 'Ġfriends']
['<|endoftext|>', 'ĠMost', 'Ġpeople', 'Ġhave', 'Ġfriends']
['<|endoftext|>', 'ĠYou', 'Ġcan', 'Ġdrive', 'Ġon', 'Ġthe', 'Ġhighway', 'Ġwithout', 'Ġa', 'Ġdriving', 'Ġlicense']
['<|endoftext|>', 'ĠYou', 'Ġcan', "'t", 'Ġdrive', 'Ġon', 'Ġthe', 'Ġhighway', 'Ġwithout', 'Ġdriving', 'Ġlicense']
['<|endoftext|>', 'ĠPeople', 'Ġdon', "'t", 'Ġneed', 'Ġto', 'Ġwear', 'Ġclothes']
['<|endoftext|>', 'ĠPeople', 'Ġneed', 'Ġto', 'Ġwear', 'Ġclothes']
['<|endoftext|>', 'ĠMost', 'Ġpeople', 'Ġforget', 'Ġthings', 'Ġsometimes']
['<|endoftext|>', 'ĠMost', 'Ġpeople', 'Ġdon', "'t", 'Ġforget', 'Ġthings', 'Ġanytime']
['<|endoftext|>', 'ĠMars', 'Ġis', 'Ġnot',

...

['<|endoftext|>', 'ĠI', 'Ġfeel', 'Ġdirty', 'Ġafter', 'Ġtaking', 'Ġa', 'Ġbath']
['<|endoftext|>', 'ĠI', 'Ġfeel', 'Ġclean', 'Ġafter', 'Ġtaking', 'Ġa', 'Ġbath']
['<|endoftext|>', 'ĠI', 'Ġwatered', 'Ġthe', 'Ġflowers', 'Ġevery', 'Ġday', ',', 'Ġso', 'Ġit', 'Ġopened', 'Ġvery', 'Ġwell']
['<|endoftext|>', 'ĠI', 'Ġdid', 'Ġnot', 'Ġwater', 'Ġthe', 'Ġflowers', ',', 'Ġso', 'Ġit', 'Ġopened', 'Ġvery', 'Ġwell']
['<|endoftext|>', 'ĠI', 'Ġwant', 'Ġto', 'Ġgo', 'Ġout', 'Ġfor', 'Ġa', 'Ġpicnic', 'Ġin', 'Ġbad', 'Ġweather']
['<|endoftext|>', 'ĠI', 'Ġwant', 'Ġto', 'Ġgo', 'Ġout', 'Ġfor', 'Ġa', 'Ġpicnic', 'Ġin', 'Ġfine', 'Ġweather']
['<|endoftext|>', 'ĠI', 'Ġwant', 'Ġto', 'Ġhave', 'Ġmy', 'Ġhair', 'Ġcut', 'Ġbecause', 'Ġmy', 'Ġhair', 'Ġis', 'Ġtoo', 'Ġlong']
['<|endoftext|>', 'ĠI', 'Ġwant', 'Ġto', 'Ġhave', 'Ġmy', 'Ġhair', 'Ġcut', 'Ġbecause', 'Ġmy', 'Ġhair', 'Ġis', 'Ġtoo', 'Ġshort']
['<|endoftext|>', 'Ġa', 'Ġlot', 'Ġof', 'Ġpeople', 'Ġget', 'Ġhurt', 'Ġin', 'Ġsleep']
['<|endoftext|>', 'Ġa', 'Ġlot', 'Ġof', 'Ġpeople', 'Ġ

['<|endoftext|>', 'ĠJim', 'Ġstole', 'Ġher', 'Ġbicycle', 'Ġand', 'Ġnow', 'Ġhe', 'Ġis', 'Ġa', 'Ġcriminal']
['<|endoftext|>', 'ĠJim', 'Ġstole', 'Ġher', 'Ġheart', 'Ġand', 'Ġnow', 'Ġhe', 'Ġis', 'Ġa', 'Ġcriminal']
['<|endoftext|>', 'Ġpeople', 'Ġsmoke', 'Ġto', 'Ġhave', 'Ġfun']
['<|endoftext|>', 'Ġpeople', 'Ġsmoke', 'Ġto', 'Ġget', 'Ġsick']
['<|endoftext|>', 'Ġthe', 'Ġdrink', 'Ġis', 'Ġtoo', 'Ġ', 'iced', 'Ġso', 'ĠI', 'Ġdropped', 'Ġmany', 'Ġice', 'Ġcubes', 'Ġin']
['<|endoftext|>', 'Ġthe', 'Ġdrink', 'Ġis', 'Ġtoo', 'Ġ', 'iced', 'Ġbecause', 'ĠI', 'Ġdropped', 'Ġmany', 'Ġice', 'Ġcubes', 'Ġin']
['<|endoftext|>', 'Ġhe', 'Ġcan', "'t", 'Ġwalk', 'Ġwith', 'Ġan', 'Ġinjured', 'Ġfoot']
['<|endoftext|>', 'Ġhe', 'Ġcan', "'t", 'Ġwalk', 'Ġwith', 'Ġan', 'Ġinjured', 'Ġhand']
['<|endoftext|>', 'Ġhe', 'Ġis', 'Ġa', 'Ġnice', 'Ġboy', 'Ġbecause', 'Ġhe', 'Ġsmiles', 'Ġto', 'Ġeveryone']
['<|endoftext|>', 'Ġhe', 'Ġis', 'Ġa', 'Ġnice', 'Ġboy', 'Ġbecause', 'Ġhe', 'Ġyells', 'Ġat', 'Ġeveryone']
['<|endoftext|>', 'Ġhe', 'Ġgot', 'Ġa

['<|endoftext|>', 'ĠI', 'Ġfelt', 'Ġsad', 'der', 'Ġafter', 'Ġeating', 'Ġterrible', 'Ġfood']
['<|endoftext|>', 'Ġmy', 'Ġbest', 'Ġfriend', 'Ġwas', 'Ġin', 'Ġmy', 'Ġlost', 'Ġwallet']
['<|endoftext|>', 'Ġmy', 'Ġfavorite', 'Ġphoto', 'Ġwas', 'Ġin', 'Ġmy', 'Ġlost', 'Ġwallet']
['<|endoftext|>', 'Ġif', 'Ġwe', 'Ġpick', 'Ġup', 'Ġother', "'s", 'Ġwallet', ',', 'Ġwe', 'Ġshould', 'Ġreturn', 'Ġit', 'Ġto', 'Ġthe', 'Ġowner']
['<|endoftext|>', 'Ġif', 'Ġwe', 'Ġpick', 'Ġup', 'Ġother', "'s", 'Ġwallet', ',', 'Ġwe', 'Ġshould', 'Ġshare', 'Ġit', 'Ġwith', 'Ġour', 'Ġfriends']
['<|endoftext|>', 'Ġmy', 'Ġfamily', 'Ġgo', 'Ġto', 'Ġthe', 'Ġpark', 'Ġto', 'Ġsee', 'Ġplum', 'Ġbloss', 'oms', 'Ġevery', 'Ġsummer']
['<|endoftext|>', 'Ġmy', 'Ġfamily', 'Ġgo', 'Ġto', 'Ġthe', 'Ġpark', 'Ġto', 'Ġsee', 'Ġlot', 'us', 'Ġevery', 'Ġsummer']
['<|endoftext|>', 'ĠI', 'Ġcan', 'Ġcount', 'Ġthe', 'Ġnumber', 'Ġof', 'Ġstars', 'Ġin', 'Ġthe', 'Ġsky']
['<|endoftext|>', 'ĠI', 'Ġcan', 'Ġcount', 'Ġthe', 'Ġnumber', 'Ġof', 'Ġthe', 'Ġmoon', 'Ġin', 'Ġthe', 

['<|endoftext|>', 'ĠI', 'Ġcan', "'t", 'Ġafford', 'Ġthe', 'Ġflight', 'Ġticket', 'Ġfrom', 'ĠChina', 'Ġto', 'Ġthe', 'ĠUS', ',', 'Ġso', 'ĠI', 'Ġdecided', 'Ġto', 'Ġtake', 'Ġship']
['<|endoftext|>', 'ĠI', 'Ġcan', "'t", 'Ġafford', 'Ġthe', 'Ġflight', 'Ġticket', 'Ġfrom', 'ĠChina', 'Ġto', 'Ġthe', 'ĠUS', ',', 'Ġso', 'ĠI', 'Ġdecided', 'Ġto', 'Ġwalk']
['<|endoftext|>', 'Ġgirls', 'Ġwear', 'Ġskirts', 'Ġin', 'Ġwinter', 'Ġfor', 'Ġbeauty']
['<|endoftext|>', 'Ġgirls', 'Ġwear', 'Ġskirts', 'Ġin', 'Ġwinter', 'Ġfor', 'Ġwarmth']
['<|endoftext|>', 'Ġdriving', 'Ġa', 'Ġcar', 'Ġis', 'Ġa', 'Ġgood', 'Ġchoice', 'Ġto', 'Ġenjoy', 'Ġyour', 'Ġstay', 'Ġat', 'Ġhome']
['<|endoftext|>', 'Ġreading', 'Ġbooks', 'Ġis', 'Ġa', 'Ġgood', 'Ġchoice', 'Ġto', 'Ġenjoy', 'Ġyour', 'Ġstay', 'Ġat', 'Ġhome']
['<|endoftext|>', 'Ġhe', 'Ġscored', 'Ġa', 'Ġcrucial', 'Ġgoal', 'Ġby', 'Ġhand', 'Ġin', 'Ġa', 'Ġsoccer', 'Ġmatch']
['<|endoftext|>', 'Ġhe', 'Ġscored', 'Ġa', 'Ġcrucial', 'Ġgoal', 'Ġby', 'Ġhead', 'Ġin', 'Ġa', 'Ġsoccer', 'Ġmatch']
['<|endofte

['<|endoftext|>', 'Ġmy', 'Ġfriend', 'Ġpaid', 'Ġfor', 'Ġmy', 'Ġticket', 'Ġso', 'ĠI', 'Ġdidn', "'t", 'Ġget', 'Ġon', 'Ġthe', 'Ġbus']
['<|endoftext|>', 'Ġd', 'ina', 'Ġvisited', 'Ġher', 'Ġfamily', 'Ġbecause', 'Ġshe', 'Ġdist', 'r', 'usted', 'Ġthem']
['<|endoftext|>', 'Ġd', 'ina', 'Ġvisited', 'Ġher', 'Ġfamily', 'Ġbecause', 'Ġshe', 'Ġmissed', 'Ġthem']
['<|endoftext|>', 'Ġthe', 'Ġballoon', 'Ġexpanded', 'Ġas', 'Ġhe', 'Ġblew', 'Ġinto', 'Ġit']
['<|endoftext|>', 'Ġthe', 'Ġballoon', 'Ġexpanded', 'Ġas', 'Ġhe', 'Ġpr', 'icked', 'Ġit']
['<|endoftext|>', 'Ġa', 'Ġhammer', 'Ġdropped', 'Ġon', 'ĠBill', "'s", 'Ġfoot', 'Ġand', 'Ġhis', 'Ġtoe', 'Ġwas', 'Ġbroken']
['<|endoftext|>', 'Ġa', 'Ġhammer', 'Ġdropped', 'Ġon', 'ĠBill', "'s", 'Ġfoot', 'Ġand', 'Ġhis', 'Ġfinger', 'Ġwas', 'Ġbroken']
['<|endoftext|>', 'Ġthe', 'Ġconversation', 'Ġgradually', 'Ġlull', 'ed', 'Ġto', 'Ġsilence', 'Ġas', 'Ġmore', 'Ġpeople', 'Ġleft']
['<|endoftext|>', 'Ġthe', 'Ġconversation', 'Ġgradually', 'Ġlull', 'ed', 'Ġto', 'Ġsilence', 'Ġas', 'Ġmore

In [175]:
wrongdf.sent

0       New York is located in the northeastern part o...
1       USA is located in the northeastern part of New...
2       his mother became satisfied after he got good ...
3       his mother became angry after he got good grad...
4              this man can jump across a mountain easily
                              ...                        
1177                         we have lunch in the library
1178                         India is a country in Europe
1179                 Great Britain is a country in Europe
1180                 you can see a dinosaur in the museum
1181                    you can see a dinosaur in the zoo
Name: sent, Length: 1182, dtype: object

In [176]:
with open('wrong_examples.txt', 'w') as f:
    for s, p in zip(wrongdf.sent, perps):
        f.write(str(p) + " " + str(s) + "\n")

In [32]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Encode a text inputs
text = "Who was Jim Henson ? Jim Henson was a"
indexed_tokens = tokenizer.encode(text)

# Convert indexed tokens in a PyTorch tensor
tokens_tensor = torch.tensor([indexed_tokens])

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json from cache at /Users/goelprat/.cache/torch/transformers/f2808208f9bec2320371a9f5f891c184ae0b674ef866b79c58177067d15732dd.1512018be4ba4e8726e41b9145129dc30651ea4fec86aa61f4b9f40bf94eac71
INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt from cache at /Users/goelprat/.cache/torch/transformers/d629f792e430b3c76a1291bb2766b0a047e36fae0588f9dbc1ae51decdff691b.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda


In [34]:
# Load pre-trained model (weights)
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set the model in evaluation mode to deactivate the DropOut modules
# This is IMPORTANT to have reproducible results during evaluation!
model.eval()

# Predict all tokens
with torch.no_grad():
    outputs = model(tokens_tensor)
    predictions = outputs[0]

# get the predicted next sub-word (in our case, the word 'man')
predicted_index = torch.argmax(predictions[0, -1, :]).item()
predicted_text = tokenizer.decode(indexed_tokens + [predicted_index])

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json from cache at /Users/goelprat/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.085d5f6a8e7812ea05ff0e6ed0645ab2e75d80387ad55c1ad9806ee70d272f80
INFO:transformers.configuration_utils:Model config {
  "attn_pdrop": 0.1,
  "embd_pdrop": 0.1,
  "finetuning_task": null,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_layer": 12,
  "n_positions": 1024,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "torchscript": false,
  "use_bfloat16": false,
  "vocab_size": 50257
}

INFO:transformers.modeling_uti

In [35]:
predicted_text

'Who was Jim Henson? Jim Henson was a man'

In [5]:
##################### Finetuning BERT using transformers library #########################

In [3]:
bertdf = df.sent.apply(lambda x : "[CLS] " + x + " [SEP]")
bertdf

0        [CLS] He poured orange juice on his cereal. [SEP]
1                [CLS] He poured milk on his cereal. [SEP]
2                             [CLS] He drinks apple. [SEP]
3                              [CLS] He drinks milk. [SEP]
4                        [CLS] Jeff ran a mile today [SEP]
                               ...                        
19995    [CLS] Harry went to the barbershop to have his...
19996          [CLS] Reilly is sleeping on the couch [SEP]
19997         [CLS] Reilly is sleeping on the window [SEP]
19998                 [CLS] I have a desk on my lamp [SEP]
19999                 [CLS] I have a lamp on my desk [SEP]
Name: sent, Length: 20000, dtype: object

In [7]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_tensors = []
for sent in df.sent:
    # Tokenize input
    tokenized_text = tokenizer.tokenize(sent)
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    train_tensors.append(tokens_tensor)

In [8]:
from torch import nn
class MyBertForSequenceClassification(nn.Module):
  
    def __init__(self, num_labels=2):
        super(MyBertForSequenceClassification, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.classifier = nn.Linear(768, num_labels)
        nn.init.xavier_normal_(self.classifier.weight)
    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask)
        logits = self.classifier(pooled_output)

        return logits

In [9]:
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertModel, BertForMaskedLM
max_seq_length = 50
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class text_dataset(Dataset):
    def __init__(self,x_y_list):
        self.x_y_list = x_y_list
        
    def __getitem__(self,index):
        tokenized_review = tokenizer.tokenize(self.x_y_list[0][index])
        if len(tokenized_review) > max_seq_length:
            tokenized_review = tokenized_review[:max_seq_length]
        ids_review  = tokenizer.convert_tokens_to_ids(tokenized_review)
        padding = [0] * (max_seq_length - len(ids_review))       
        ids_review += padding      
        assert len(ids_review) == max_seq_length       
        ids_review = torch.tensor(ids_review)        
        sentiment = self.x_y_list[1][index] # color        
        list_of_labels = [torch.from_numpy(np.array(sentiment))]
        return ids_review, list_of_labels[0]
    
    def __len__(self):
        return len(self.x_y_list[0])

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/goelprat/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [32]:
import torch
import time
import os
import copy
import torch.nn.functional as F
import numpy as np

def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    print('starting')
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 100

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            sentiment_corrects = 0
                        
            # Iterate over data.
            for inputs, targets in dataloaders_dict[phase]: 
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    outputs = F.softmax(outputs,dim=1)  
                    loss = criterion(outputs, targets.long())
                    # backward + optimize only if in training phase
                    if phase == 'train':                        
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                print('Running loss : {:.4f}'.format(running_loss))
                sentiment_corrects += torch.sum(torch.max(outputs, 1)[1] == targets.long())
            epoch_loss = running_loss / dataset_sizes[phase]
            sentiment_acc = sentiment_corrects.double() / dataset_sizes[phase]
            print('{} total loss: {:.4f} '.format(phase,epoch_loss ))
            print('{} sentiment_acc: {:.4f}'.format(
                phase, sentiment_acc))

            if phase == 'val' and epoch_loss < best_loss:
                print('saving with loss of {}'.format(epoch_loss),
                      'improved over previous {}'.format(best_loss))
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save(model.state_dict(), 'bert_model_test.pth')
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    #print('Best val Acc: {:4f}'.format(float(best_loss)))
    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [34]:
batch_size = 16
train_lists = [df.sent[:1000], df.isvalid[:1000]]
test_lists = [testdf.sent[:100], testdf.isvalid[:100]]
training_dataset = text_dataset(x_y_list = train_lists)
test_dataset = text_dataset(x_y_list = test_lists)

dataloaders_dict = {'train': torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=0),
                   'val':torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
                   }
dataset_sizes = {'train':len(train_lists[0]),
                'val':len(test_lists[0])}

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [17]:
model = MyBertForSequenceClassification()

In [31]:
from torch import optim
from torch.optim import lr_scheduler
lrlast = .001
lrmain = .00001
optim1 = optim.Adam(
    [
        {"params":model.bert.parameters(),"lr": lrmain},
        {"params":model.classifier.parameters(), "lr": lrlast},
       
   ])

#optim1 = optim.Adam(model.parameters(), lr=0.001)#,momentum=.9)
# Observe that all parameters are being optimized
optimizer_ft = optim1
criterion = nn.CrossEntropyLoss()

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=3, gamma=0.1)

In [ ]:
model_ft1 = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=5)

starting
Epoch 0/4
----------
Running loss : 12.3019
Running loss : 23.1574
Running loss : 34.4371
Running loss : 47.0812
Running loss : 58.0687
Running loss : 69.9240
Running loss : 79.8553
Running loss : 90.6626
Running loss : 98.7249
Running loss : 108.6554
Running loss : 121.1563
Running loss : 133.6704
Running loss : 147.1135
Running loss : 160.5545
Running loss : 173.0695
Running loss : 187.0734
Running loss : 200.6931
Running loss : 212.4535
Running loss : 225.4815
Running loss : 236.5354
Running loss : 247.6441
Running loss : 259.3732
Running loss : 270.5484
Running loss : 281.8232
Running loss : 292.8590
Running loss : 303.2048
Running loss : 314.9358
Running loss : 324.8700
Running loss : 335.8258
Running loss : 347.7164
Running loss : 359.6016
Running loss : 370.9968
Running loss : 382.9237
Running loss : 393.9334
Running loss : 404.9101
Running loss : 416.2926
Running loss : 427.3474
Running loss : 438.9927
Running loss : 450.3670
Running loss : 461.2991
Running loss : 472.

In [20]:
test_dataset

In [27]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
results = []
for inputs, targets in test_loader:
    outputs = model_ft1(inputs)
    outputs = F.softmax(outputs,dim=1)
    for i in range(len(outputs)):
        predicted_index = torch.argmax(outputs[i]).item()
        results.append(predicted_index)
    print("Loop done")

Loop done
Loop done
Loop done
Loop done
Loop done
Loop done
Loop done


In [78]:
import math
def get_score(tokens_tensor):
    predictions = None
    # Predict all tokens
    loss_fct = torch.nn.CrossEntropyLoss()
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]
        loss = loss_fct(predictions.squeeze(),tokens_tensor.squeeze()).data 
        return math.exp(loss)

tokens_tensor = train_tensors[12]
print(get_score(tokens_tensor))
tokens_tensor = train_tensors[13]
print(get_score(tokens_tensor))

760.5078798734681
298.18919185123605


In [108]:
testdf.sent

0                      he put an elephant into the fridge
1                         he put a turkey into the fridge
2       my sister eats an apple after breakfast every day
3        my sister eats a stone after breakfast every day
4                       money can be used for buying cars
                              ...                        
4037    people have to hold onto their shoes because o...
4038    Rebecca wakes up because she takes sleeping pills
4039           Rebecca wakes up because she sets an alarm
4040                      people bleed when they feel hot
4041                      people sweat when they feel hot
Name: sent, Length: 4042, dtype: object

In [51]:
tokens_tensor.squeeze()

tensor([  101,  2002,  8542,  6501,  2006,  2010, 20943,  1012,   102])

In [128]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)
transdf = testdf.sent.apply(lambda x : "[CLS] " + x + " [SEP]")
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

test_tensors = []
for sent in transdf:
    # Tokenize input
    tokenized_text = tokenizer.tokenize(sent)
    # Convert token to vocabulary indices
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    test_tensors.append(tokens_tensor)

INFO:transformers.tokenization_utils:loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/goelprat/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [136]:
transdf[3]

'[CLS] my sister eats a stone after breakfast every day [SEP]'

In [132]:
outputs = model(test_tensors[3])
predictions = outputs[0]
predictions = predictions.squeeze()
predicted_tokens = []
for i in range(len(predictions)):
    predicted_index = torch.argmax(predictions[i]).item()
    predicted_tokens.append(tokenizer.convert_ids_to_tokens([predicted_index])[0])
predicted_tokens

['.',
 '.',
 'sister',
 'eats',
 'a',
 'stone',
 'after',
 'breakfast',
 'every',
 'day',
 '.']

In [97]:
results = []
for i in range(0, len(test_tensors), 2):
    t1 = test_tensors[i]
    t2 = test_tensors[i + 1]
    s1 = get_score(t1)
    s2 = get_score(t2)
    if s1 <= s2:
        results.append(1)
    else:
        results.append(0)

KeyboardInterrupt: 

In [88]:
x_test.id

0          1
1          2
2          3
3          4
4          5
        ... 
2016    2017
2017    2018
2018    2019
2019    2020
2020    2021
Name: id, Length: 2021, dtype: int64

In [127]:
pd.concat([x_test.id, pd.Series(results)], axis=1).to_csv('./predictions.csv', header=False, index=False)

In [ ]:
!python eval/taskA_scorer.py --gold-labels data/train/subtaskA_answers_all.csv --pred-labels predictions.csv

In [74]:
!python eval/taskA_scorer.py --gold-labels data/trial/taskA_trial_answer.csv --pred-labels predictions.csv

  File "eval/taskA_scorer.py", line 24
    def calculate_accuracy(gold_labels: Dict[str, str], predictions: Dict[str, List[str]]) -> float:
                                      ^
SyntaxError: invalid syntax


In [22]:
!source ~/miniconda3/bin/activate fastai